In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

import pandas as pd

import seaborn as sns

from my_functions import *

import glob
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67

In [ ]:
## Load my QSO catalog

filename = '/home/alberto/almacen/Source_cats/QSO_100000_v7/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data_qso = pd.concat(fi, axis=0, ignore_index=True)

qso_flx = data_qso.to_numpy()[:, 1 : 60 + 1].T
qso_err = data_qso.to_numpy()[:, 60 + 1 : 120 + 1].T

EW_qso = data_qso['EW0'].to_numpy()
qso_zspec = data_qso['z'].to_numpy()

In [ ]:
## Load my GAL catalog

filename = '/home/alberto/almacen/Source_cats/GAL_100000/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data_gal = pd.concat(fi, axis=0, ignore_index=True)

gal_flx = data_gal.to_numpy()[:, 1 : 60 + 1].T
gal_err = data_gal.to_numpy()[:, 60 + 1 : 120 + 1].T

EW_gal = np.zeros(data_gal['z'].to_numpy().shape)
gal_zspec = data_gal['z'].to_numpy()

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-4_v7/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T

EW_sf = data['EW0'].to_numpy()
sf_zspec = data['z'].to_numpy()

In [ ]:
pm_flx = np.hstack((qso_flx, sf_flx, gal_flx))
pm_err = np.hstack((qso_err, sf_err, gal_err))
zspec = np.concatenate((qso_zspec, sf_zspec, gal_zspec))
EW_lya = np.concatenate((EW_qso, EW_sf, EW_gal))

N_sf = sf_flx.shape[1]
N_qso = qso_flx.shape[1]
N_gal = gal_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value
sf_dL = cosmo.luminosity_distance(sf_zspec).to(u.cm).value

sf_L = data['L_lya'].to_numpy()
qso_L = data_qso['L_lya'].to_numpy()
gal_L = np.zeros(EW_gal.shape)

sf_flambda = 10 ** sf_L / (4*np.pi * sf_dL **2)
qso_flambda = data_qso['F_line']
gal_flambda = np.zeros(EW_gal.shape)

L_lya = np.concatenate((qso_L, sf_L, gal_L))
fline = np.concatenate((qso_flambda, sf_flambda, gal_flambda))

is_qso = np.concatenate((np.ones(N_qso), np.zeros(N_sf + N_gal))).astype(bool)
is_sf = np.concatenate((np.zeros(N_qso), np.ones(N_sf), np.zeros(N_gal))).astype(bool)
is_gal = np.concatenate((np.zeros(N_qso), np.zeros(N_sf), np.ones(N_gal))).astype(bool)

N_sources = pm_flx.shape[1]

%xdel sf_flx
%xdel sf_err
%xdel qso_flx
%xdel qso_err
%xdel gal_flx
%xdel gal_err
%xdel sf_zspec
%xdel qso_zspec
%xdel gal_zspec
%xdel EW_sf
%xdel EW_qso
%xdel EW_gal
%xdel qso_dL
%xdel sf_L
%xdel qso_L
%xdel sf_flambda
%xdel qso_flambda
%xdel gal_flambda
%xdel mock
%xdel data
%xdel data_qso
%xdel data_gal

In [ ]:
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 20)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, pm_err, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    400, obs=True)
other_lines = identify_lines(line_other, pm_flx, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.12

# %xdel cont_est_other
# %xdel cont_err_other

In [ ]:
ew0lya_min = 0
ew0lya_max = 70
ew0lya_step = 8
ew0oth_min = 0
ew0oth_max = 400
ew0oth_step = 11

other_select_list = []
for ew0min in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step):
    print(ew0min)
    line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other, ew0min,
        obs=True)
    other_lines = identify_lines(line_other, pm_flx, pm_err, first=False)

    other_select_list.append(other_lines)

lya_select_list = []
lya_z_nb = []
for ew0min in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step):
    print(ew0min)
    line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min)
    lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)
    z_nb_Arr = np.ones(N_sources) * -1 
    z_nb_Arr[np.where(np.array(lya_lines) != -1)] =\
        z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])
    
    lya_select_list.append(lya_lines)
    lya_z_nb.append(z_nb_Arr)

In [ ]:
mag_cut = 24
z_cut_min = 2.45
z_cut_max = 3.54

select_grid = np.zeros((ew0lya_step, ew0oth_step))
rightz_grid = np.zeros((ew0lya_step, ew0oth_step))

target_z = (zspec > z_cut_min) & (zspec < z_cut_max)
target = (
    (mag < mag_cut) & target_z & (EW_lya > 20) & (L_lya > 44)
)

gal_factor = 12.57

for i in range(ew0lya_step):
    print(i)
    for j in range(ew0oth_step):
        nice_lya = (
            nice_lya_select(
                lya_select_list[i], other_select_list[j], pm_flx, pm_err,
                cont_est_other, lya_z_nb[i]
            )
        )
        nice_z = np.abs(lya_z_nb[i] - zspec) < 0.12
        
        z_nb_cut = (lya_z_nb[i] > z_cut_min) & (lya_z_nb[i] < z_cut_max)

        selected = (mag < mag_cut) & nice_lya & z_nb_cut & (is_qso | is_sf)
        selected_gal = (mag < mag_cut) & nice_lya & z_nb_cut & is_gal
        select_grid[i, j] = count_true(selected)
        select_grid[i, j] += count_true(selected_gal) * gal_factor
        rightz_grid[i, j] = len(np.where(selected & nice_z)[0])
        print(count_true(selected_gal))

In [ ]:
target = (
    (mag < mag_cut) & target_z & (EW_lya > 20) & (L_lya > 44)
)

In [ ]:
N_target = len(np.where(target)[0])
purity = rightz_grid / select_grid
completeness = rightz_grid / N_target

In [ ]:
fig = plt.figure(figsize=(8, 8))

width = 0.5
height = 0.5
spacing = 0.06
cbar_width = 0.05

# Define axes
ax00 = fig.add_axes([0, height + 1.5 * spacing, width, height])
ax01 = fig.add_axes([width + spacing, height + 1.5 * spacing, width, height], sharey=ax00)
ax10 = fig.add_axes([0, 0, width, height], sharex=ax00)
ax11 = fig.add_axes([width + spacing, 0, width, height], sharex=ax01, sharey=ax10)
axcbar0 = fig.add_axes([2 * width + 1.5 * spacing, height + 1.5 * spacing, cbar_width, height])
axcbar1 = fig.add_axes([2 * width + 1.5 * spacing, 0, cbar_width, height])

# Plot stuff in the rectangles
vmax = np.max([np.max(rightz_grid), np.max(select_grid)])
cmap = 'Spectral'

sns.heatmap(rightz_grid, ax=ax00, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)
sns.heatmap(select_grid, ax=ax01, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)

sns.heatmap(purity, ax=ax10, vmin=0, vmax=1, cbar_ax=axcbar1, cmap=cmap)
sns.heatmap(completeness, ax=ax11, vmin=0, vmax=1, cbar=False, cmap=cmap)

ax00.invert_yaxis()
ax10.invert_yaxis()

# Axes ticks
xticks = range(ew0oth_step)
yticks = range(ew0lya_step)
xtick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step)]
ytick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step)]

ax00.set_yticks(yticks)
ax00.set_yticklabels(ytick_labels)
ax00.set_xticks(xticks)
ax00.set_xticklabels(xtick_labels, rotation='vertical')

ax10.set_yticks(yticks)
ax10.set_yticklabels(ytick_labels)
ax10.set_xticks(xticks)
ax10.set_xticklabels(xtick_labels, rotation='vertical')

ax11.set_xticks(xticks)
ax11.set_xticklabels(xtick_labels, rotation='vertical')
ax11.set_yticks(yticks)
ax11.set_yticklabels(ytick_labels)

ax01.set_xticks(xticks)
ax01.set_xticklabels(xtick_labels, rotation='vertical')
ax01.set_yticks(yticks)
ax01.set_yticklabels(ytick_labels)

# Axes labels
ylabel = r'Ly$\alpha$ EW$_0$ ($\AA$)'
xlabel = 'Other lines EW$_\mathrm{obs}$ ($\AA$)'
ax00.set_ylabel(ylabel, fontsize=12)
ax10.set_ylabel(ylabel, fontsize=12)
ax10.set_xlabel(xlabel, fontsize=12)
ax11.set_xlabel(xlabel, fontsize=12)

# Set titles
ax00.set_title('Selected w/ correct z', fontsize=15)
ax01.set_title('All selected', fontsize=15)
ax10.set_title('Purity', fontsize=15)
ax11.set_title('Completeness', fontsize=15)

plt.savefig('output/puri-comp_magcut-' + str(mag_cut) + '.pdf', dpi=600,
    bbox_inches='tight')
plt.show()

print('N_target = {}'.format(N_target))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

puricomp_norm = (purity ** 2 + completeness ** 2) ** 0.5
sns.heatmap(puricomp_norm, ax=ax, cmap=cmap)
print(np.argmax(puricomp_norm))

ax.set_yticks(yticks)
ax.set_yticklabels(ytick_labels)
ax.set_xticks(xticks)
ax.set_xticklabels(xtick_labels, rotation='vertical')

ax.invert_yaxis()

ax.set_ylabel(ylabel, fontsize=12)
ax.set_xlabel(xlabel, fontsize=12)

ax.set_title(r'$\sqrt{\mathrm{Purity}^2 + \mathrm{Completeness}^2}$', fontsize=20)

plt.show()